In [7]:
# =====================================================================
# 3×3  H(α) = M₀ + α·diag(m)      –  eigen-values, polynomial, vectors
# =====================================================================
import numpy as np, plotly.graph_objects as go, ipywidgets as W
from plotly.subplots import make_subplots
from IPython.display import display
import sympy as sp

α_grid = np.linspace(-5,5,801); nα=len(α_grid)

pars = dict(m11=0.0, m22=1.0, m33=2.0,
            M0_12=1.0, M0_13=1.0, M0_23=1.0,
            M0_11=0.0, M0_22=1.0, M0_33=2.0)

# ── matrices
def M1(p): return np.diag([p['m11'],p['m22'],p['m33']])
def M0(p):
    return np.array([[p['M0_11'],p['M0_12'],p['M0_13']],
                     [p['M0_12'],p['M0_22'],p['M0_23']],
                     [p['M0_13'],p['M0_23'],p['M0_33']]],float)

# ── tracks
def eigen_tracks(p):
    M0_,M1_=M0(p),M1(p)
    return np.array([np.linalg.eigvalsh(M0_+a*M1_) for a in α_grid])

def eigenvec_tracks(p):
    M0_,M1_=M0(p),M1(p)
    V=np.empty((nα,3,3))
    _,V0=np.linalg.eigh(M0_+α_grid[0]*M1_); V[0]=V0
    for i in range(1,nα):
        _,Vcur=np.linalg.eigh(M0_+α_grid[i]*M1_)
        for j in range(3):
            if np.dot(Vcur[:,j],V[i-1,:,j])<0: Vcur[:,j]*=-1
        V[i]=Vcur
    return V

def asymptotes(p): return np.sort(np.diag(M1(p)))

# ── symbolic p(λ,α)
λsym,αsym=sp.symbols('λ α')
_sup=str.maketrans('0123456789-','⁰¹²³⁴⁵⁶⁷⁸⁹⁻')
def cstr(x): return f"{x:+.3g}"
def fmt_poly(expr):
    poly=sp.Poly(expr,λsym,αsym); out=[]
    for (i,j),c in sorted(poly.terms(),key=lambda t:(-sum(t[0]),t[0])):
        if abs(c)<1e-12: continue
        t=cstr(c)
        if j: t+=f" α{'' if j==1 else str(j).translate(_sup)}"
        if i: t+=f" λ{'' if i==1 else str(i).translate(_sup)}"
        out.append(t)
    return " ".join(out).lstrip('+').replace('+ -',' - ')

def sym_poly_expr(p):
    H=sp.Matrix(M0(p))+αsym*sp.Matrix(M1(p))
    return sp.expand(H.charpoly(λsym).as_expr())

# ── pre-compute first time
tracks      = eigen_tracks(pars)
vec_tracks  = eigenvec_tracks(pars)
Λ           = asymptotes(pars)
poly_disp   = fmt_poly(sym_poly_expr(pars))

# ── figure (3 stacked panels)
fig=make_subplots(rows=3,cols=1,
                  specs=[[{"type":"xy"}],[{"type":"xy"}],[{"type":"scene"}]],
                  row_heights=[0.45,0.30,0.25],vertical_spacing=0.07,
                  subplot_titles=("Eigen-value tracks  λᵢ(α)",
                                  "Characteristic polynomial  pₐ(λ)",
                                  "Eigen-vector directions"))
fig=go.FigureWidget(fig)

eig_cols=['#1f77b4','#ff7f0e','#00cc96']; dash='rgba(120,120,120,0.6)'
WIN=[-15,15]

# panel-1: values & asymptotes
for j,c in enumerate(eig_cols):
    fig.add_scatter(x=α_grid,y=tracks[:,j],line=dict(color=c,width=3),
                    name=f'λ{j+1}',row=1,col=1)
for j in range(3):
    fig.add_scatter(x=α_grid,y=α_grid*Λ[j],
                    line=dict(color=dash,dash='dash'),showlegend=False,
                    row=1,col=1)
fig.add_scatter(x=[0,0],y=WIN,mode='lines',
                line=dict(color='rgba(255,0,0,0.35)',width=4),
                showlegend=False,row=1,col=1)

# panel-2: polynomial & roots
λ_plot=np.linspace(-20,20,1601)
fig.add_scatter(x=λ_plot,y=np.zeros_like(λ_plot),
                line=dict(color='black'),showlegend=False,row=2,col=1)
for c in eig_cols:
    fig.add_scatter(x=[0],y=[0],mode='markers',
                    marker=dict(size=10,color=c),showlegend=False,row=2,col=1)




####################################### ONLY BLOCK 3 I SEE ################################################
# ────────────────────────────────────────────────────────────────────────
# panel-3: dashed vector tracks (thicker) + cones + spheres
# ────────────────────────────────────────────────────────────────────────
# 1) static dashed trajectories — traces 11,12,13
for j,c in enumerate(eig_cols):
    fig.add_scatter3d(
        x=vec_tracks[:,j,0],
        y=vec_tracks[:,j,1],
        z=vec_tracks[:,j,2],
        mode='lines',
        line=dict(color=c,  width=4),
        showlegend=False,
        row=3, col=1
    )

# 2) arrow shafts placeholders — traces 14,15,16
for j,c in enumerate(eig_cols):
    fig.add_scatter3d(
        x=[0,0], y=[0,0], z=[0,0],
        mode='lines',
        line=dict(color=c, width=6),
        showlegend=False,
        row=3, col=1
    )

# 3) cones for heads — traces 17,18,19
for j,c in enumerate(eig_cols):
    fig.add_trace(go.Cone(
        x=[0], y=[0], z=[0],
        u=[0], v=[0], w=[0],
        showscale=False,
        colorscale=[[0,c],[1,c]],
        sizemode='absolute',
        sizeref=0.3,
        anchor='tail'
    ), row=3, col=1)

# 4) spheres at tips — traces 20,21,22
for c in eig_cols:
    fig.add_scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers',
        marker=dict(size=5, color=c),
        showlegend=False,
        row=3, col=1
    )

####################################### END OF THE ONLY BLOCK 3 I SEE ################################################

# fixed cube
lim=[-1.2,1.2]
fig.update_layout(width=700,height=1300,
                  margin=dict(l=6,r=6,t=60,b=6),
                  legend=dict(orientation='h',x=1.02,y=1.12,xanchor='right'),
                  font=dict(size=13),
                  scene=dict(aspectmode='cube',
                             xaxis=dict(range=lim,autorange=False,title='x'),
                             yaxis=dict(range=lim,autorange=False,title='y'),
                             zaxis=dict(range=lim,autorange=False,title='z')))

fig.update_xaxes(range=[-5,5],title='α',row=1,col=1)
fig.update_yaxes(range=WIN,title='λ',row=1,col=1)
fig.update_xaxes(range=[-20,20],title='λ',row=2,col=1)
fig.update_yaxes(range=[-50,50],title='pₐ(λ)',row=2,col=1)

# ── widgets
slider_cfg=dict(min=-20,max=20,step=0.01,continuous_update=True,
                layout=W.Layout(width='300px'),
                style=dict(description_width='54px'))
pretty=dict(m11="M¹₁₁",m22="M¹₂₂",m33="M¹₃₃",
            M0_11="M⁰₁₁",M0_22="M⁰₂₂",M0_33="M⁰₃₃",
            M0_12="M⁰₁₂",M0_13="M⁰₁₃",M0_23="M⁰₂₃")
param_sliders={k:W.FloatSlider(value=v,description=pretty[k],**slider_cfg)
               for k,v in pars.items()}
α_slider=W.FloatSlider(value=0,min=-5,max=5,step=0.01,continuous_update=True,
                       description='α',readout_format='.3f',
                       layout=W.Layout(width='690px'))
info_box=W.HTML(layout=W.Layout(width='310px'))
round3=lambda v:f"{v:+.3f}"

# heavy recompute
def recompute():
    global tracks,Λ,vec_tracks,poly_disp
    tracks=eigen_tracks(pars)
    Λ=asymptotes(pars)
    vec_tracks=eigenvec_tracks(pars)
    poly_disp=fmt_poly(sym_poly_expr(pars))

# callbacks
def on_matrix(_):
    for k,s in param_sliders.items(): pars[k]=s.value
    recompute()
    with fig.batch_update():
        for j in range(3):
            fig.data[j].y=tracks[:,j]
            fig.data[j+3].y=α_grid*Λ[j]
            t=11+j
            fig.data[t].x,fig.data[t].y,fig.data[t].z = \
                vec_tracks[:,j,0],vec_tracks[:,j,1],vec_tracks[:,j,2]
    on_alpha({'new':α_slider.value})

def on_alpha(change):
    a   = change['new']
    idx = np.clip(np.searchsorted(α_grid, a), 0, nα-1)
    eig = tracks[idx]
    vec = vec_tracks[idx]
    coeff = np.poly(M0(pars) + a*M1(pars))

    with fig.batch_update():
        # ——— update α-bar ———
        fig.data[6].x = [a, a]

        # ——— update polynomial curve ———
        fig.data[7].y = np.polyval(coeff, λ_plot)

        # ——— update roots on λ-axis ———
        for j in range(3):
            fig.data[8 + j].x = [eig[j]]
            fig.data[8 + j].y = [0]

        # ——— update the three arrow-shafts ———
        #    (these are the straight lines from origin to vec[j])
        for j in range(3):
            u, v, w = vec[j]
            shaft_trace = 14 + j
            fig.data[shaft_trace].x = [0, u]
            fig.data[shaft_trace].y = [0, v]
            fig.data[shaft_trace].z = [0, w]

        # ——— update the cones (arrow-heads) ———
        for j in range(3):
            u, v, w = vec[j]
            cone_trace = 17 + j
            fig.data[cone_trace].x = [u]
            fig.data[cone_trace].y = [v]
            fig.data[cone_trace].z = [w]
            fig.data[cone_trace].u = [u]
            fig.data[cone_trace].v = [v]
            fig.data[cone_trace].w = [w]

        # ——— update the spheres at the tip ———
        for j in range(3):
            u, v, w = vec[j]
            sphere_trace = 20 + j
            fig.data[sphere_trace].x = [u]
            fig.data[sphere_trace].y = [v]
            fig.data[sphere_trace].z = [w]

    # ——— update the info box below ———
    coeff_text = (f"{coeff[0]:+.3g} λ³ "
                  f"{coeff[1]:+.3g} λ² "
                  f"{coeff[2]:+.3g} λ "
                  f"{coeff[3]:+.3g}")

    info_box.value = f"""
<pre style="font-family:monospace">
M₀ =
{np.array2string(M0(pars),precision=2,suppress_small=True)}

M₁ =
{np.array2string(M1(pars),precision=2,suppress_small=True)}

p(λ,α)      = {poly_disp}
pₐ(λ)       = {coeff_text}

α           = {a:+.3f}
Λ (asympt.) = [{', '.join(f'{x:+.3f}' for x in Λ)}]
λ (tracks)  = [{', '.join(f'{x:+.3f}' for x in eig)}]
</pre>"""

for s in param_sliders.values(): s.observe(on_matrix,'value')
α_slider.observe(on_alpha,'value')

recompute(); on_alpha({'new':0.0})

left=W.VBox(list(param_sliders.values())+[info_box],
            layout=W.Layout(width='330px'))
display(W.VBox([W.HBox([left,fig],layout=W.Layout(align_items='flex-start')),
                α_slider]))
